In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from Bio import SeqIO
import numpy as np
import re
from collections import defaultdict
import argparse
import os

def parse_fasta(fasta_file):
    """Parse a FASTA file and return a dictionary of {protein_name: sequence}"""
    protein_dict = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        protein_dict[record.id] = str(record.seq)
    return protein_dict

def map_peptides_to_proteins(peptide_csv, protein_dict):
    """
    Map peptides to their positions in proteins.
    
    Args:
        peptide_csv: Path to CSV file with columns [peptides, Source, HLA types]
        protein_dict: Dictionary of {protein_name: sequence}
    
    Returns:
        Dictionary with mapping information
    """
    # Read peptide data
    df = pd.read_csv(peptide_csv)
    
    # Initialize results structure
    mapping = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    
    # Process each peptide
    for _, row in df.iterrows():
        peptide = row['peptides']
        source = row['Source']
        hla_type = row['HLA types']
        
        # Determine MHC class from HLA type
        mhc_class = "MHC-I" if hla_type.startswith(('HLA-A', 'HLA-B', 'HLA-C')) else "MHC-II"
        
        # Determine section (default to Current - Ligand if not specified)
        section = "Current - Ligand"
        if 'Section' in row:
            section = row['Section']
        
        # Find matching protein(s)
        for protein_name, protein_seq in protein_dict.items():
            # Check if protein name matches the source
            if source.lower() in protein_name.lower() or protein_name.lower() in source.lower():
                # Find all occurrences of peptide in protein
                for match in re.finditer(peptide, protein_seq):
                    start_pos = match.start()
                    end_pos = match.end()
                    
                    # Store mapping information
                    mapping[section][protein_name][mhc_class].append({
                        'peptide': peptide,
                        'start': start_pos,
                        'end': end_pos,
                        'hla': hla_type
                    })
    
    return mapping

def extract_gene_names(proteins):
    """Extract gene names from protein IDs"""
    gene_names = {}
    for protein in proteins:
        # Extract gene name from protein ID format: >sp|ID|NAME OS=X OX=Y GN=GeneName
        match = re.search(r'GN=(\w+)', protein)
        if match:
            gene_name = match.group(1)
            gene_names[protein] = gene_name
        else:
            # If GN tag not found, try to extract name from the ID
            parts = protein.split('|')
            if len(parts) > 2:
                name_parts = parts[2].split(' ')
                gene_names[protein] = name_parts[0]
            else:
                gene_names[protein] = protein
    
    return gene_names

def visualize_peptide_mapping(mapping, protein_dict, output_file="peptide_mapping.png", custom_order=None):
    """
    Create visualization of peptide mappings similar to the image.
    
    Args:
        mapping: Dictionary with mapping information
        protein_dict: Dictionary of {protein_name: sequence}
        output_file: Path to save the output image
        custom_order: Optional list specifying order of protein display
    """
    # Get unique proteins
    proteins = set()
    for section in mapping:
        for protein in mapping[section]:
            proteins.add(protein)
    proteins = list(proteins)
    
    # Get gene names
    gene_names = extract_gene_names(proteins)
    
    # Standard colors
    colors = {
        'IEDB - Epitopes': {'MHC-I': '#ffcc66', 'MHC-II': '#ffcc66'},
        'IEDB - Ligand': {'MHC-I': '#999999', 'MHC-II': '#000000'},
        'Current - Ligand': {'MHC-I': '#ff6666', 'MHC-II': '#ff6666'}
    }
    
    # Calculate protein lengths for scaling
    protein_lengths = {protein: len(protein_dict[protein]) for protein in proteins}
    max_length = max(protein_lengths.values())
    
    # Setup figure
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Title
    plt.suptitle('Peptide Mapping to Proteins', fontsize=14)
    
    # Track vertical position
    y_pos = 10
    
    # Define protein colors based on gene names
    protein_colors = {}
    for protein, gene in gene_names.items():
        # Assign colors based on gene name patterns or types
        if gene.startswith(('Auts', 'Iqcj', 'Arr')):
            protein_colors[gene] = '#d17895'  # Pink/purple for certain gene families
        elif gene.startswith(('Fc', 'Gpr')):
            protein_colors[gene] = '#c06c5e'  # Red-ish for receptor proteins
        elif gene.startswith(('G', 'I')):
            protein_colors[gene] = '#5e9c6c'  # Green for other proteins
        else:
            protein_colors[gene] = '#aaaaaa'  # Default grey
    
    # Sections to draw
    sections = ['IEDB - Epitopes', 'IEDB - Ligand', 'Current - Ligand']
    
    # Only include sections that have data
    sections = [s for s in sections if s in mapping]
    
    # Draw sections
    for section in sections:
        # Add section label
        ax.text(0.5, y_pos + 2, section, ha='center', fontsize=12)
        y_pos += 4
        
        # Add MHC class labels and mappings
        for mhc_class in ['MHC-II', 'MHC-I']:
            ax.text(0.05, y_pos, mhc_class, ha='left', va='center', fontsize=10)
            
            # For each protein, draw peptide mappings
            for protein in proteins:
                if section in mapping and protein in mapping[section] and mhc_class in mapping[section][protein]:
                    for peptide_info in mapping[section][protein][mhc_class]:
                        start = peptide_info['start'] / max_length
                        width = (peptide_info['end'] - peptide_info['start']) / max_length
                        
                        # Scale to figure width (leaving margins)
                        start = 0.1 + start * 0.8
                        width = width * 0.8
                        
                        # Draw rectangle for peptide
                        rect = patches.Rectangle(
                            (start, y_pos - 0.3), 
                            width, 
                            0.6, 
                            facecolor=colors[section][mhc_class],
                            alpha=0.8
                        )
                        ax.add_patch(rect)
            
            y_pos += 2
        
        # Add separator
        ax.axhline(y=y_pos, color='lightgray', linestyle='--', alpha=0.7)
        y_pos += 2
    
    # Add protein regions at the bottom
    y_pos += 2
    ax.text(0.5, y_pos, "Proteins", ha='center', fontsize=12)
    y_pos += 2
    
    # Calculate protein widths based on length
    total_width = 0.8  # 80% of plot width
    margin = 0.1  # 10% margin on each side
    
    # Order proteins
    if custom_order:
        # Try to match custom order with gene names
        ordered_genes = []
        for name in custom_order:
            for protein, gene in gene_names.items():
                if name.lower() in gene.lower():
                    ordered_genes.append(protein)
                    break
        # Add any remaining proteins
        ordered_genes.extend([p for p in proteins if p not in ordered_genes])
    else:
        # Use alphabetical order by gene name
        ordered_genes = sorted(proteins, key=lambda p: gene_names[p])
    
    # Calculate positions
    gene_positions = {}
    x_start = margin
    
    for protein in ordered_genes:
        gene = gene_names[protein]
        length = protein_lengths[protein]
        width = (length / max_length) * total_width
        gene_positions[gene] = (x_start, width)
        x_start += width
    
    # Draw protein regions
    for protein in ordered_genes:
        gene = gene_names[protein]
        x_start, width = gene_positions[gene]
        
        # Draw protein rectangle
        color = protein_colors.get(gene, '#aaaaaa')
        rect = patches.Rectangle(
            (x_start, y_pos - 0.5),
            width,
            1,
            facecolor=color,
            alpha=0.8
        )
        ax.add_patch(rect)
        
        # Add protein label
        ax.text(
            x_start + width/2,
            y_pos + 1,
            gene,
            ha='center',
            va='center',
            fontsize=10
        )
    
    # Set plot limits and remove axes
    ax.set_xlim(0, 1)
    ax.set_ylim(0, y_pos + 3)
    ax.axis('off')
    
    # Save figure
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Visualization saved to {output_file}")

def create_sample_csv(output_file="sample_peptides.csv"):
    """Create a sample CSV file with the expected format"""
    sample_data = """peptides,Source,HLA types,Section
STFIYNSM,Gbp6,HLA-DQA1*05:01,IEDB - Epitopes
CLQPPDGGT,Iqcj-Schip1,HLA-B*07:02,IEDB - Epitopes
PLGSTVQSQTKGIWM,Gbp6,HLA-DQB1*02:01,IEDB - Ligand
SDDYLENAL,Gbp6,HLA-A*02:01,Current - Ligand
SARVKDSG,Fcgr4,HLA-DQA1*05:01,Current - Ligand
HPQLPQLPS,Auts2,HLA-A*02:01,IEDB - Epitopes
LPERDGHSHEGRAAG,Auts2,HLA-B*07:02,IEDB - Ligand
SGSDKDSDAD,Iqcj-Schip1,HLA-A*02:01,Current - Ligand
CVRAVLERP,Arrdc4,HLA-B*35:01,Current - Ligand
HNDSGSYFCRGL,Fcgr4,HLA-DRB1*01:01,IEDB - Epitopes"""
    
    with open(output_file, 'w') as f:
        f.write(sample_data)
    
    print(f"Sample CSV created at {output_file}")

# def main():
    # """Command-line interface for the peptide mapping tool"""
    # parser = argparse.ArgumentParser(description='Map peptides to proteins and visualize the mappings.')
    # parser.add_argument('fasta_file', help='Path to FASTA file with protein sequences')
    # parser.add_argument('peptide_csv', help='Path to CSV file with peptide information')
    # parser.add_argument('-o', '--output', default='peptide_mapping.png', help='Output image file path')
    # parser.add_argument('--order', nargs='+', help='Custom order for proteins display')
    # parser.add_argument('--sample', action='store_true', help='Create a sample CSV file')
    
    # args = parser.parse_args()
    
    # if args.sample:
    #     create_sample_csv()
    #     return
    
    # # Validate input files
    # if not os.path.exists(args.fasta_file):
    #     print(f"Error: FASTA file '{args.fasta_file}' not found")
    #     return
    
    # if not os.path.exists(args.peptide_csv):
    #     print(f"Error: CSV file '{args.peptide_csv}' not found")
    #     return
    
    # Parse proteins
    # print(f"Parsing proteins from {args.fasta_file}")
    # protein_dict = parse_fasta(args.fasta_file)
    # print(f"Found {len(protein_dict)} proteins")
    
    # # Map peptides to proteins
    # print(f"Mapping peptides from {args.peptide_csv}")
    # mapping = map_peptides_to_proteins(args.peptide_csv, protein_dict)
    
    # # Create visualization
    # print("Creating visualization")
    # visualize_peptide_mapping(mapping, protein_dict, args.output, args.order)
    
    # print(f"Process completed. Visualization saved to {args.output}")


def main():
    create_sample_csv()
    # Example usage
    fasta_file = "sample_peptides.fasta"  # Path to your FASTA file
    peptide_csv = "peptides.csv"  # Path to your peptide CSV file
    output_file = "peptide_mapping.png"  # Output image file name
    custom_order = None  # Optional custom order for protein display
    
    # Parse proteins
    protein_dict = parse_fasta(fasta_file)
    
    # Map peptides to proteins
    mapping = map_peptides_to_proteins(peptide_csv, protein_dict)
    
    # Create visualization
    visualize_peptide_mapping(mapping, protein_dict, output_file, custom_order)
if __name__ == "__main__":
    main()

Sample CSV created at sample_peptides.csv
Visualization saved to peptide_mapping.png


In [17]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re
from Bio import SeqIO
from io import StringIO
import numpy as np

def parse_fasta(fasta_file):
    """Parse FASTA file and return a dictionary of {protein_id: sequence}"""
    protein_dict = {}
    protein_info = {}
    
    with open(fasta_file, 'r') as file:
        for record in SeqIO.parse(file, "fasta"):
            # Get protein ID
            id_parts = record.id.split('|')
            if len(id_parts) > 1:
                protein_id = id_parts[1].split('_')[0]  # Get base protein ID
            else:
                protein_id = record.id
            
            # Get protein name from description
            match = re.search(r'GN=(\w+)', record.description)
            if match:
                protein_name = match.group(1)
            else:
                protein_name = protein_id
            
            # Store in dictionaries
            protein_dict[protein_id] = str(record.seq)
            protein_info[protein_id] = {
                'name': protein_name,
                'description': record.description,
                'length': len(record.seq)
            }
    
    return protein_dict, protein_info

def parse_peptides(peptides_file):
    """Parse peptide file and return a list of peptides"""
    peptides = []
    with open(peptides_file, 'r') as file:
        for line in file:
            peptide = line.strip()
            if peptide:  # Skip empty lines
                peptides.append(peptide)
    return peptides

def map_peptides_to_proteins(peptides, protein_dict):
    """Map peptides to their positions in proteins"""
    mapping = {}
    
    for protein_id, sequence in protein_dict.items():
        mapping[protein_id] = []
        
        for peptide in peptides:
            # Find all occurrences of the peptide in the protein
            start = 0
            while True:
                start = sequence.find(peptide, start)
                if start == -1:
                    break
                end = start + len(peptide)
                mapping[protein_id].append({
                    'peptide': peptide,
                    'start': start,
                    'end': end
                })
                start += 1  # Move to next position to continue searching
    
    return mapping

def visualize_peptide_distribution(mapping, protein_info, output_file="peptide_distribution.png"):
    """Create visualization of peptide mappings"""
    # Filter out proteins with no peptides
    mapping = {k: v for k, v in mapping.items() if v}
    
    if not mapping:
        print("No peptides found in any proteins.")
        return
    
    # Setup figure
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Title
    plt.suptitle('Peptide Distribution Across Proteins', fontsize=14)
    
    # Track vertical position
    y_pos = 3
    
    # Add MHC label
    ax.text(0.05, y_pos, "MHC", ha='left', va='center', fontsize=10)
    
    # Define protein colors
    protein_colors = {
        'PB1': '#d17895',    # Pink-ish
        'NS1': '#c06c5e',    # Red-ish
        'M1': '#5e9c6c',     # Green-ish
        'HA': '#5e9cbc',     # Blue-ish
        'NA': '#bc5e9c',     # Purple-ish
        'NP': '#9cbc5e',     # Yellow-ish
        'PA': '#9c5ebc',     # Violet-ish
        'PB2': '#5ebc9c'     # Cyan-ish
    }
    
    # Find max protein length for scaling
    max_length = max(info['length'] for info in protein_info.values() if info['name'] in [protein_info[pid]['name'] for pid in mapping.keys()])
    
    # Process peptide mapping
    for protein_id in mapping.keys():
        if not mapping[protein_id]:  # Skip proteins with no peptides
            continue
        
        protein_name = protein_info[protein_id]['name']
        protein_length = protein_info[protein_id]['length']
        
        # Draw peptide mappings
        for peptide_info in mapping[protein_id]:
            start = peptide_info['start'] / max_length
            width = (peptide_info['end'] - peptide_info['start']) / max_length
            
            # Scale to figure width (leaving margins)
            start = 0.2 + start * 0.6
            width = width * 0.6
            
            # Draw rectangle for peptide
            rect = patches.Rectangle(
                (start, y_pos - 0.3),
                width,
                0.6,
                facecolor='#ff6666',  # Red for peptides
                alpha=0.8
            )
            ax.add_patch(rect)
    
    # Move down for protein regions
    y_pos -= 2
    
    # Add separator
    ax.axhline(y=y_pos, color='lightgray', linestyle='--', alpha=0.7)
    y_pos -= 1
    
    # Calculate positions for proteins
    total_width = 0.6  # 60% of plot width
    margin = 0.2      # 20% margin on each side
    x_start = margin
    
    # Draw protein regions
    for protein_id in mapping.keys():
        if not mapping[protein_id]:  # Skip proteins with no peptides
            continue
        
        protein_name = protein_info[protein_id]['name']
        protein_length = protein_info[protein_id]['length']
        width = (protein_length / max_length) * total_width
        
        # Draw protein rectangle
        color = protein_colors.get(protein_name, '#aaaaaa')
        rect = patches.Rectangle(
            (x_start, y_pos - 0.5),
            width,
            1,
            facecolor=color,
            alpha=0.8
        )
        ax.add_patch(rect)
        
        # Add protein label
        ax.text(
            x_start + width/2,
            y_pos,
            protein_name,
            ha='center',
            va='center',
            fontsize=10
        )
        
        x_start += width
    
    # Set plot limits and remove axes
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 4)
    ax.axis('off')
    
    # Save figure
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Visualization saved to {output_file}")

def main(fasta_file, peptides_file, output_file="peptide_distribution.png"):
    """Main function to run the entire process"""
    # Parse input files
    print(f"Parsing proteins from {fasta_file}")
    protein_dict, protein_info = parse_fasta(fasta_file)
    print(f"Found {len(protein_dict)} proteins")
    
    print(f"Parsing peptides from {peptides_file}")
    peptides = parse_peptides(peptides_file)
    print(f"Found {len(peptides)} peptides")
    
    # Map peptides to proteins
    print("Mapping peptides to proteins")
    mapping = map_peptides_to_proteins(peptides, protein_dict)
    
    # Count mapped peptides for each protein
    for protein_id, peptide_list in mapping.items():
        if peptide_list:
            print(f"  {protein_info[protein_id]['name']}: {len(peptide_list)} peptide mappings")
    
    # Create visualization
    print("Creating visualization")
    visualize_peptide_distribution(mapping, protein_info, output_file)
    
    print(f"Process completed. Visualization saved to {output_file}")

if __name__ == "__main__":
    # import sys
    
    # if len(sys.argv) < 3:
    #     print("Usage: python script.py X31_proteomes.fasta peptides.txt [output_file.png]")
    #     sys.exit(1)
    
    # fasta_file = sys.argv[1]
    # peptides_file = sys.argv[2]
    
    # if len(sys.argv) > 3:
    #     output_file = sys.argv[3]
    pep = "/Users/sanjay/Monash/Master_thesis/lab_work/Li_Lab/ProtPeptigram/example/new_tools/peptides.txt"
    prot = "/Users/sanjay/Monash/Master_thesis/lab_work/Li_Lab/ProtPeptigram/example/new_tools/X31_proteomes.fasta"
    main(prot,pep, output_file="peptide_distribution.png")
    # else:
    #     main(fasta_file, peptides_file)

Parsing proteins from /Users/sanjay/Monash/Master_thesis/lab_work/Li_Lab/ProtPeptigram/example/new_tools/X31_proteomes.fasta
Found 10 proteins
Parsing peptides from /Users/sanjay/Monash/Master_thesis/lab_work/Li_Lab/ProtPeptigram/example/new_tools/peptides.txt
Found 81 peptides
Mapping peptides to proteins
  PB2: 14 peptide mappings
  NS: 5 peptide mappings
  NP: 23 peptide mappings
  HA: 7 peptide mappings
  NA: 1 peptide mappings
  NS: 4 peptide mappings
Creating visualization
Visualization saved to peptide_distribution.png
Process completed. Visualization saved to peptide_distribution.png


In [20]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re
from Bio import SeqIO
from io import StringIO
import numpy as np
import argparse
import os

def parse_fasta(fasta_file):
    """Parse FASTA file and return a dictionary of {protein_id: sequence}"""
    protein_dict = {}
    protein_info = {}
    
    with open(fasta_file, 'r') as file:
        for record in SeqIO.parse(file, "fasta"):
            # Extract protein ID from header
            header_parts = record.id.split('|')
            if len(header_parts) > 1:
                protein_id = header_parts[1]
            else:
                protein_id = record.id
            
            # Get protein name from description
            description = record.description
            
            # Try to extract GN= part
            match = re.search(r'GN=(\w+)', description)
            if match:
                gene_name = match.group(1)
            else:
                # If no GN tag, use the part after Protein keyword
                match = re.search(r'Protein\s+([^\s]+)', description)
                if match:
                    gene_name = match.group(1)
                else:
                    gene_name = protein_id
            
            # Store in dictionaries
            protein_dict[protein_id] = str(record.seq)
            protein_info[protein_id] = {
                'name': gene_name,
                'description': description,
                'length': len(record.seq),
                'id': protein_id
            }
    
    return protein_dict, protein_info

def parse_peptides(peptides_file):
    """Parse peptide file and return a list of peptides"""
    peptides = []
    with open(peptides_file, 'r') as file:
        for line in file:
            line = line.strip()
            if line and not line.startswith('#'):  # Skip empty lines and comments
                peptides.append(line)
    return peptides

def map_peptides_to_proteins(peptides, protein_dict):
    """Map peptides to their positions in proteins"""
    mapping = {}
    
    for protein_id, sequence in protein_dict.items():
        mapping[protein_id] = []
        
        for peptide in peptides:
            # Find all occurrences of the peptide in the protein
            start = 0
            while True:
                start = sequence.find(peptide, start)
                if start == -1:
                    break
                end = start + len(peptide)
                mapping[protein_id].append({
                    'peptide': peptide,
                    'start': start,
                    'end': end
                })
                start += 1  # Move to next position to continue searching
    
    return mapping

def get_simplified_proteins(protein_info):
    """Group proteins by gene name to simplify visualization"""
    simplified = {}
    
    for protein_id, info in protein_info.items():
        gene_name = info['name']
        if gene_name not in simplified:
            simplified[gene_name] = {
                'length': info['length'],
                'proteins': [protein_id],
                'description': info['description']
            }
        else:
            simplified[gene_name]['proteins'].append(protein_id)
            if info['length'] > simplified[gene_name]['length']:
                simplified[gene_name]['length'] = info['length']
    
    return simplified

def visualize_peptide_distribution(mapping, protein_info, output_file="peptide_distribution.png"):
    """Create visualization of peptide mappings similar to the reference image"""
    # Get simplified protein view (by gene name)
    simplified_proteins = get_simplified_proteins(protein_info)
    
    # Filter to include only proteins with peptides
    proteins_with_peptides = set()
    for protein_id, peptides in mapping.items():
        if peptides:
            gene_name = protein_info[protein_id]['name']
            proteins_with_peptides.add(gene_name)
    
    # Keep only proteins with peptides
    simplified_proteins = {k: v for k, v in simplified_proteins.items() if k in proteins_with_peptides}
    
    if not simplified_proteins:
        print("No peptides found in any proteins.")
        return
    
    # Setup figure
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Title
    plt.suptitle('Peptide Distribution Across Influenza Proteins', fontsize=14)
    
    # Track vertical position
    y_pos = 5
    
    # Add section label
    ax.text(0.5, y_pos + 1, "Current - Ligand", ha='center', fontsize=12)
    y_pos -= 1
    
    # Add MHC label
    ax.text(0.05, y_pos, "MHC", ha='left', va='center', fontsize=10)
    
    # Define protein colors (similar to the reference image)
    protein_colors = {
        'PB1': '#d17895',    # Pink-ish
        'NS1': '#c06c5e',    # Red-ish
        'M1': '#5e9c6c',     # Green-ish
        'HA': '#d17895',     # Same as NSP1
        'NA': '#d17895',     # Same as NSP4
        'NP': '#d17895',     # Same as NSP8
        'PA': '#d17895',     # Same as NSP9
        'PB2': '#c06c5e'     # Same as Envelope
    }
    
    # Default color for other proteins
    default_color = '#5e9c6c'  # Same as Nucleoprotein
    
    # Find max protein length for scaling
    max_length = max(info['length'] for info in simplified_proteins.values())
    
    # Process peptide mapping
    for gene_name, gene_info in simplified_proteins.items():
        for protein_id in gene_info['proteins']:
            if protein_id not in mapping or not mapping[protein_id]:
                continue
                
            for peptide_info in mapping[protein_id]:
                start = peptide_info['start'] / max_length
                width = (peptide_info['end'] - peptide_info['start']) / max_length
                
                # Scale to figure width (leaving margins)
                start = 0.1 + start * 0.8
                width = max(width * 0.8, 0.005)  # Ensure minimum visible width
                
                # Draw rectangle for peptide
                rect = patches.Rectangle(
                    (start, y_pos - 0.3),
                    width,
                    0.6,
                    facecolor='#ff6666',  # Red for peptides
                    alpha=0.8
                )
                ax.add_patch(rect)
    
    # Move down for protein regions
    y_pos -= 2
    
    # Add separator
    ax.axhline(y=y_pos, color='lightgray', linestyle='--', alpha=0.7)
    y_pos -= 1
    
    # Calculate positions for proteins based on length
    total_width = 0.8  # 80% of plot width
    margin = 0.1      # 10% margin on each side
    x_start = margin
    
    # Sort proteins by name to ensure consistent order
    sorted_genes = sorted(simplified_proteins.keys())
    
    # Draw protein regions
    for gene_name in sorted_genes:
        gene_info = simplified_proteins[gene_name]
        protein_length = gene_info['length']
        width = (protein_length / max_length) * total_width
        
        # Draw protein rectangle
        color = protein_colors.get(gene_name, default_color)
        rect = patches.Rectangle(
            (x_start, y_pos - 0.5),
            width,
            1,
            facecolor=color,
            alpha=0.8
        )
        ax.add_patch(rect)
        
        # Add protein label
        ax.text(
            x_start + width/2,
            y_pos,
            gene_name,
            ha='center',
            va='center',
            fontsize=10
        )
        
        x_start += width
    
    # Set plot limits and remove axes
    ax.set_xlim(0, 1)
    ax.set_ylim(y_pos - 1, 7)
    ax.axis('off')
    
    # Save figure
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Visualization saved to {output_file}")

def print_peptide_mappings(mapping, protein_info):
    """Print detailed information about peptide mappings"""
    print("\nDetailed Peptide Mappings:")
    print("--------------------------")
    
    for protein_id, peptides in mapping.items():
        if not peptides:
            continue
            
        gene_name = protein_info[protein_id]['name']
        print(f"\n{gene_name} ({protein_id}):")
        
        # Group by peptide
        peptide_groups = {}
        for p_info in peptides:
            peptide = p_info['peptide']
            if peptide not in peptide_groups:
                peptide_groups[peptide] = []
            peptide_groups[peptide].append(p_info)
        
        # Print each peptide with its positions
        for peptide, positions in peptide_groups.items():
            print(f"  Peptide: {peptide}")
            for pos in positions:
                print(f"    Position: {pos['start']}-{pos['end']} (1-indexed: {pos['start']+1}-{pos['end']})")

def main():
    # """Main function with argument parsing"""
    # parser = argparse.ArgumentParser(description='Map peptides to proteins and visualize the distribution.')
    # parser.add_argument('fasta_file', help='Path to FASTA file with protein sequences')
    # parser.add_argument('peptides_file', help='Path to file with peptide sequences (one per line)')
    # parser.add_argument('-o', '--output', default='peptide_distribution.png', help='Output image file path')
    # parser.add_argument('-d', '--detail', action='store_true', help='Print detailed mapping information')
    
    # args = parser.parse_args()
    
    # # Validate input files
    # if not os.path.exists(args.fasta_file):
    #     print(f"Error: FASTA file '{args.fasta_file}' not found")
    #     return
    
    # if not os.path.exists(args.peptides_file):
    #     print(f"Error: Peptides file '{args.peptides_file}' not found")
    #     return
    
    # # Parse input files
    # print(f"Parsing proteins from {args.fasta_file}")
    # protein_dict, protein_info = parse_fasta(args.fasta_file)
    # print(f"Found {len(protein_dict)} proteins")
    
    # print(f"Parsing peptides from {args.peptides_file}")
    # peptides = parse_peptides(args.peptides_file)
    # print(f"Found {len(peptides)} peptides")
    
    # # Map peptides to proteins
    # print("Mapping peptides to proteins")
    # mapping = map_peptides_to_proteins(peptides, protein_dict)
    
    # # Count mapped peptides for each protein
    # mapped_count = 0
    # for protein_id, peptide_list in mapping.items():
    #     if peptide_list:
    #         gene_name = protein_info[protein_id]['name']
    #         print(f"  {gene_name} ({protein_id}): {len(peptide_list)} peptide mappings")
    #         mapped_count += len(peptide_list)
    
    # print(f"Total: {mapped_count} peptide mappings")
    
    # if args.detail:
    #     print_peptide_mappings(mapping, protein_info)
    
    # # Create visualization
    # print("Creating visualization")
    # visualize_peptide_distribution(mapping, protein_info, args.output)
    
    # print(f"Process completed. Visualization saved to {args.output}")
    protein_dict, protein_info = parse_fasta("X31_proteomes.fasta")
    # Parse peptides
    peptides = parse_peptides("peptides.txt")
    # Map peptides to proteins
    mapping = map_peptides_to_proteins(peptides, protein_dict)
    # # Count mapped peptides for each protein
    mapped_count = 0
    for protein_id, peptide_list in mapping.items():
        if peptide_list:
            gene_name = protein_info[protein_id]['name']
            print(f"  {gene_name} ({protein_id}): {len(peptide_list)} peptide mappings")
            mapped_count += len(peptide_list)
    print_peptide_mappings(mapping, protein_info)
    visualize_peptide_distribution(mapping, protein_info, "peptide_distribution.png")
if __name__ == "__main__":
    main()

  PB2 (P03428): 14 peptide mappings
  HKx31_HA_translation_frame_+3 (HKx31_HA_translation_frame_+3): 7 peptide mappings
  PR8_PA_translation_frame_+1 (PR8_PA_translation_frame_+1): 4 peptide mappings
  NS (P03508_26E): 5 peptide mappings
  M (P03485): 8 peptide mappings
  NP (P03466): 23 peptide mappings
  HA (Q91MA7_198I): 7 peptide mappings
  PR8_M_translation_frame_+2 (PR8_M_translation_frame_+2): 8 peptide mappings
  PR8_PB1_translation_frame_+1 (PR8_PB1_translation_frame_+1): 9 peptide mappings
  PR8_PB2_translation_frame_+1 (PR8_PB2_translation_frame_+1): 16 peptide mappings
  PR8_NS_translation_frame_+1 (PR8_NS_translation_frame_+1): 6 peptide mappings
  PR8_NS_translation_frame_+3 (PR8_NS_translation_frame_+3): 4 peptide mappings
  NA (Q91MA2): 1 peptide mappings
  PR8_NP_translation_frame_-1 (PR8_NP_translation_frame_-1): 9 peptide mappings
  HKx31_NA_translation_frame_+1 (HKx31_NA_translation_frame_+1): 1 peptide mappings
  NS (P03496_55K): 4 peptide mappings
  HKx31_HA_trans

/var/folders/qj/qvj3dgfx3v5bcz5lx3fxn8l00000gn/T/ipykernel_39622/2802312402.py:230: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Visualization saved to peptide_distribution.png


In [24]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import pairwise2
from io import StringIO
import numpy as np
import argparse
import os
import csv

def parse_fasta(fasta_file):
    """Parse FASTA file and return a dictionary of {protein_id: sequence}"""
    protein_dict = {}
    protein_info = {}
    
    with open(fasta_file, 'r') as file:
        for record in SeqIO.parse(file, "fasta"):
            # Extract protein ID from header
            header_parts = record.id.split('|')
            if len(header_parts) > 1:
                protein_id = header_parts[1]
            else:
                protein_id = record.id
            
            # Get protein name from description
            description = record.description
            
            # Try to extract GN= part
            match = re.search(r'GN=(\w+)', description)
            if match:
                gene_name = match.group(1)
            else:
                # If no GN tag, use the part after Protein keyword
                match = re.search(r'Protein\s+([^\s]+)', description)
                if match:
                    gene_name = match.group(1)
                else:
                    gene_name = protein_id
            
            # Store in dictionaries
            protein_dict[protein_id] = str(record.seq)
            protein_info[protein_id] = {
                'name': gene_name,
                'description': description,
                'length': len(record.seq),
                'id': protein_id
            }
    
    return protein_dict, protein_info

def parse_peptides(peptides_file):
    """Parse peptide file and return a list of peptides"""
    peptides = []
    with open(peptides_file, 'r') as file:
        for line in file:
            line = line.strip()
            if line and not line.startswith('#'):  # Skip empty lines and comments
                peptides.append(line)
    return peptides

def calculate_mutations(peptide, matched_sequence):
    """Calculate mutations between peptide and matched sequence"""
    mutations = []
    for i, (p, m) in enumerate(zip(peptide, matched_sequence)):
        if p != m:
            mutations.append(f"{m}{i+1}{p}")  # Format: original_position_new
    return ", ".join(mutations) if mutations else ""

def find_peptide_matches(peptide, sequence, max_mutations=2):
    """Find peptide matches in sequence with up to max_mutations differences"""
    matches = []
    
    # Length of peptide
    peptide_len = len(peptide)
    
    # Skip if peptide is longer than sequence
    if peptide_len > len(sequence):
        return matches
    
    # First, try exact match (faster)
    start = 0
    while True:
        start = sequence.find(peptide, start)
        if start == -1:
            break
        end = start + peptide_len
        matches.append({
            'peptide': peptide,
            'start': start,
            'end': end,
            'matched_seq': peptide,
            'mutations': ""
        })
        start += 1
    
    # If max_mutations > 0, use alignment to find inexact matches
    if max_mutations > 0 and not matches:  # Only if exact match wasn't found
        # Slide a window along the sequence
        for i in range(len(sequence) - peptide_len + 1):
            window = sequence[i:i+peptide_len]
            
            # Count mismatches
            mismatches = sum(1 for a, b in zip(peptide, window) if a != b)
            
            if mismatches <= max_mutations:
                mutations = calculate_mutations(peptide, window)
                matches.append({
                    'peptide': peptide,
                    'start': i,
                    'end': i + peptide_len,
                    'matched_seq': window,
                    'mutations': mutations
                })
    
    return matches

def map_peptides_to_proteins(peptides, protein_dict, max_mutations=2, allow_mutations=True):
    """Map peptides to their positions in proteins with optional mutations"""
    mapping = {}
    
    for protein_id, sequence in protein_dict.items():
        mapping[protein_id] = []
        
        for peptide in peptides:
            if allow_mutations:
                matches = find_peptide_matches(peptide, sequence, max_mutations)
            else:
                matches = find_peptide_matches(peptide, sequence, 0)  # Only exact matches
            
            mapping[protein_id].extend(matches)
    
    return mapping

def get_top_proteins(mapping, protein_info, top_n=4):
    """Get the top N proteins with the most peptide matches"""
    # Count matches per protein
    protein_counts = {}
    for protein_id, matches in mapping.items():
        if matches:
            protein_counts[protein_id] = len(matches)
    
    # Sort by count (descending)
    sorted_proteins = sorted(protein_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Get top N (or fewer if there aren't enough)
    top_proteins = sorted_proteins[:top_n]
    
    # Return as a set of protein_ids
    return {pid for pid, _ in top_proteins}

def get_simplified_proteins(protein_info, selected_proteins=None):
    """Group proteins by gene name to simplify visualization"""
    simplified = {}
    
    for protein_id, info in protein_info.items():
        # Skip if not in selected proteins
        if selected_proteins and protein_id not in selected_proteins:
            continue
            
        gene_name = info['name']
        if gene_name not in simplified:
            simplified[gene_name] = {
                'length': info['length'],
                'proteins': [protein_id],
                'description': info['description']
            }
        else:
            simplified[gene_name]['proteins'].append(protein_id)
            if info['length'] > simplified[gene_name]['length']:
                simplified[gene_name]['length'] = info['length']
    
    return simplified

def visualize_peptide_distribution(mapping, protein_info, output_file="peptide_distribution.png", top_n=4):
    """Create visualization of peptide mappings for top N proteins"""
    # Get top proteins
    top_proteins = get_top_proteins(mapping, protein_info, top_n)
    
    # Filter mapping to only include top proteins
    filtered_mapping = {k: v for k, v in mapping.items() if k in top_proteins}
    
    # Get simplified protein view (by gene name)
    simplified_proteins = get_simplified_proteins(protein_info, top_proteins)
    
    if not simplified_proteins:
        print("No peptides found in any proteins.")
        return
    
    # Setup figure
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Title
    plt.suptitle('Peptide Distribution Across Top Influenza Proteins', fontsize=14)
    
    # Track vertical position
    y_pos = 5
    
    # Add section label
    ax.text(0.5, y_pos + 1, "Current - Ligand", ha='center', fontsize=12)
    y_pos -= 1
    
    # Add MHC label
    ax.text(0.05, y_pos, "MHC", ha='left', va='center', fontsize=10)
    
    # Define protein colors (similar to the reference image)
    protein_colors = {
        'PB1': '#d17895',    # Pink-ish
        'NS1': '#c06c5e',    # Red-ish
        'M1': '#5e9c6c',     # Green-ish
        'HA': '#d17895',     # Same as NSP1
        'NA': '#d17895',     # Same as NSP4
        'NP': '#d17895',     # Same as NSP8
        'PA': '#d17895',     # Same as NSP9
        'PB2': '#c06c5e'     # Same as Envelope
    }
    
    # Default color for other proteins
    default_color = '#5e9c6c'  # Same as Nucleoprotein
    
    # Find max protein length for scaling
    max_length = max(info['length'] for info in simplified_proteins.values())
    
    # Process peptide mapping
    for gene_name, gene_info in simplified_proteins.items():
        for protein_id in gene_info['proteins']:
            if protein_id not in filtered_mapping or not filtered_mapping[protein_id]:
                continue
                
            for peptide_info in filtered_mapping[protein_id]:
                start = peptide_info['start'] / max_length
                width = (peptide_info['end'] - peptide_info['start']) / max_length
                
                # Scale to figure width (leaving margins)
                start = 0.1 + start * 0.8
                width = max(width * 0.8, 0.005)  # Ensure minimum visible width
                
                # Draw rectangle for peptide
                rect = patches.Rectangle(
                    (start, y_pos - 0.3),
                    width,
                    0.6,
                    facecolor='#ff6666',  # Red for peptides
                    alpha=0.8
                )
                ax.add_patch(rect)
    
    # Move down for protein regions
    y_pos -= 2
    
    # Add separator
    ax.axhline(y=y_pos, color='lightgray', linestyle='--', alpha=0.7)
    y_pos -= 1
    
    # Calculate positions for proteins based on length
    total_width = 0.8  # 80% of plot width
    margin = 0.1      # 10% margin on each side
    x_start = margin
    
    # Sort proteins by name to ensure consistent order
    sorted_genes = sorted(simplified_proteins.keys())
    
    # Draw protein regions
    for gene_name in sorted_genes:
        gene_info = simplified_proteins[gene_name]
        protein_length = gene_info['length']
        width = (protein_length / max_length) * total_width
        
        # Draw protein rectangle
        color = protein_colors.get(gene_name, default_color)
        rect = patches.Rectangle(
            (x_start, y_pos - 0.5),
            width,
            1,
            facecolor=color,
            alpha=0.8
        )
        ax.add_patch(rect)
        
        # Add protein label
        ax.text(
            x_start + width/2,
            y_pos,
            gene_name,
            ha='center',
            va='center',
            fontsize=10
        )
        
        x_start += width
    
    # Set plot limits and remove axes
    ax.set_xlim(0, 1)
    ax.set_ylim(y_pos - 1, 7)
    ax.axis('off')
    
    # Save figure
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Visualization saved to {output_file}")
    
    return simplified_proteins

def export_csv(mapping, protein_info, csv_file="peptide_mappings.csv"):
    """Export peptide mappings to a CSV file"""
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        
        # Write header
        writer.writerow(['peptide', 'source_protein', 'start', 'end', 'matched_sequence', 'mutations'])
        
        # Write data rows
        for protein_id, matches in mapping.items():
            if not matches:
                continue
                
            gene_name = protein_info[protein_id]['name']
            source = f"{gene_name} ({protein_id})"
            
            for match in matches:
                writer.writerow([
                    match['peptide'],
                    source,
                    match['start'] + 1,  # Convert to 1-indexed for readability
                    match['end'],
                    match['matched_seq'],
                    match['mutations']
                ])
    
    print(f"CSV export saved to {csv_file}")

def print_peptide_mappings(mapping, protein_info):
    """Print detailed information about peptide mappings"""
    print("\nDetailed Peptide Mappings:")
    print("--------------------------")
    
    for protein_id, matches in mapping.items():
        if not matches:
            continue
            
        gene_name = protein_info[protein_id]['name']
        print(f"\n{gene_name} ({protein_id}):")
        
        # Group by peptide
        peptide_groups = {}
        for m_info in matches:
            peptide = m_info['peptide']
            if peptide not in peptide_groups:
                peptide_groups[peptide] = []
            peptide_groups[peptide].append(m_info)
        
        # Print each peptide with its positions
        for peptide, positions in peptide_groups.items():
            print(f"  Peptide: {peptide}")
            for pos in positions:
                # Show mutations if present
                mutation_info = f" Mutations: {pos['mutations']}" if pos['mutations'] else ""
                print(f"    Position: {pos['start']}-{pos['end']} (1-indexed: {pos['start']+1}-{pos['end']}){mutation_info}")

def main(fasta_file="X31_proteomes.fasta", peptides_file="peptides.txt", output_png="peptide_distribution.png", 
         output_csv="peptide_mappings.csv", max_mutations=0, allow_mutations=False, top_n=4, print_details=True):
    """Run the peptide mapping process with all options"""
    # Validate input files
    if not os.path.exists(fasta_file):
        print(f"Error: FASTA file '{fasta_file}' not found")
        return
    
    if not os.path.exists(peptides_file):
        print(f"Error: Peptides file '{peptides_file}' not found")
        return
    
    # Parse input files
    print(f"Parsing proteins from {fasta_file}")
    protein_dict, protein_info = parse_fasta(fasta_file)
    print(f"Found {len(protein_dict)} proteins")
    
    print(f"Parsing peptides from {peptides_file}")
    peptides = parse_peptides(peptides_file)
    print(f"Found {len(peptides)} peptides")
    
    # Map peptides to proteins
    print(f"Mapping peptides to proteins (max mutations: {max_mutations if allow_mutations else 0})")
    mapping = map_peptides_to_proteins(peptides, protein_dict, max_mutations, allow_mutations)
    
    # Count mapped peptides for each protein
    mapped_count = 0
    for protein_id, matches in mapping.items():
        if matches:
            gene_name = protein_info[protein_id]['name']
            print(f"  {gene_name} ({protein_id}): {len(matches)} peptide mappings")
            mapped_count += len(matches)
    
    print(f"Total: {mapped_count} peptide mappings")
    
    # Print detailed information if requested
    if print_details:
        print_peptide_mappings(mapping, protein_info)
    
    # Create visualization for top proteins
    print(f"Creating visualization for top {top_n} proteins")
    simplified_proteins = visualize_peptide_distribution(mapping, protein_info, output_png, top_n)
    
    # Export to CSV
    print("Exporting mappings to CSV")
    export_csv(mapping, protein_info, output_csv)
    
    print(f"Process completed.")
    print(f"- Visualization saved to {output_png}")
    print(f"- CSV data saved to {output_csv}")
    
    # Return data for further use if needed
    return mapping, protein_info, simplified_proteins

if __name__ == "__main__":
    # Use argparse for command-line usage
    # parser = argparse.ArgumentParser(description='Map peptides to proteins and visualize the distribution.')
    # parser.add_argument('fasta_file', nargs='?', default="X31_proteomes.fasta", help='Path to FASTA file with protein sequences')
    # parser.add_argument('peptides_file', nargs='?', default="peptides.txt", help='Path to file with peptide sequences (one per line)')
    # parser.add_argument('-o', '--output', default='peptide_distribution.png', help='Output image file path')
    # parser.add_argument('-c', '--csv', default='peptide_mappings.csv', help='Output CSV file path')
    # parser.add_argument('-m', '--mutations', type=int, default=2, help='Maximum number of mutations allowed (default: 2)')
    # parser.add_argument('-e', '--exact', action='store_true', help='Only find exact peptide matches')
    # parser.add_argument('-t', '--top', type=int, default=4, help='Visualize only the top N proteins with most matches (default: 4)')
    # parser.add_argument('-d', '--detail', action='store_true', help='Print detailed mapping information')
    
    # args = parser.parse_args()
    
    # Run the main function with parsed arguments
    # main(
    #     fasta_file=args.fasta_file,
    #     peptides_file=args.peptides_file,
    #     output_png=args.output,
    #     output_csv=args.csv,
    #     max_mutations=args.mutations,
    #     allow_mutations=not args.exact,
    #     top_n=args.top,
    #     print_details=args.detail
    # )
    main()

Parsing proteins from X31_proteomes.fasta
Found 56 proteins
Parsing peptides from peptides.txt
Found 81 peptides
Mapping peptides to proteins (max mutations: 0)
  PB2 (P03428): 14 peptide mappings
  HKx31_HA_translation_frame_+3 (HKx31_HA_translation_frame_+3): 7 peptide mappings
  PR8_PA_translation_frame_+1 (PR8_PA_translation_frame_+1): 4 peptide mappings
  NS (P03508_26E): 5 peptide mappings
  M (P03485): 8 peptide mappings
  NP (P03466): 23 peptide mappings
  HA (Q91MA7_198I): 7 peptide mappings
  PR8_M_translation_frame_+2 (PR8_M_translation_frame_+2): 8 peptide mappings
  PR8_PB1_translation_frame_+1 (PR8_PB1_translation_frame_+1): 9 peptide mappings
  PR8_PB2_translation_frame_+1 (PR8_PB2_translation_frame_+1): 16 peptide mappings
  PR8_NS_translation_frame_+1 (PR8_NS_translation_frame_+1): 6 peptide mappings
  PR8_NS_translation_frame_+3 (PR8_NS_translation_frame_+3): 4 peptide mappings
  NA (Q91MA2): 1 peptide mappings
  PR8_NP_translation_frame_-1 (PR8_NP_translation_frame_-

/var/folders/qj/qvj3dgfx3v5bcz5lx3fxn8l00000gn/T/ipykernel_39622/1515881298.py:299: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Visualization saved to peptide_distribution.png
Exporting mappings to CSV
CSV export saved to peptide_mappings.csv
Process completed.
- Visualization saved to peptide_distribution.png
- CSV data saved to peptide_mappings.csv


In [25]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import re
from Bio import SeqIO
import os
import csv

def parse_fasta(fasta_file):
    """Parse FASTA file and return dictionaries of protein sequences and info"""
    protein_dict = {}
    protein_info = {}
    
    with open(fasta_file, 'r') as file:
        for record in SeqIO.parse(file, "fasta"):
            # Extract protein ID
            header_parts = record.id.split('|')
            if len(header_parts) > 1:
                protein_id = header_parts[1]
            else:
                protein_id = record.id
            
            # Get protein name
            match = re.search(r'GN=(\w+)', record.description)
            if match:
                gene_name = match.group(1)
            else:
                match = re.search(r'Protein\s+([^\s]+)', record.description)
                if match:
                    gene_name = match.group(1)
                else:
                    gene_name = protein_id
            
            # Store protein information
            protein_dict[protein_id] = str(record.seq)
            protein_info[protein_id] = {
                'name': gene_name,
                'length': len(record.seq),
                'id': protein_id
            }
    
    return protein_dict, protein_info

def parse_peptides(peptides_file):
    """Parse peptide file and return a list of peptides"""
    peptides = []
    with open(peptides_file, 'r') as file:
        for line in file:
            line = line.strip()
            if line and not line.startswith('#'):
                peptides.append(line)
    return peptides

def find_peptide_matches(peptides, protein_dict):
    """Find all peptide matches in proteins"""
    mapping = {}
    
    for protein_id, sequence in protein_dict.items():
        mapping[protein_id] = []
        
        for peptide in peptides:
            # Find all occurrences of the peptide
            start = 0
            while True:
                start = sequence.find(peptide, start)
                if start == -1:
                    break
                end = start + len(peptide)
                mapping[protein_id].append({
                    'peptide': peptide,
                    'start': start,
                    'end': end
                })
                start += 1
    
    return mapping

def create_stacked_visualization(mapping, protein_dict, protein_info, output_file="peptide_stacked_alignment.png"):
    """Create a visualization with proteins stacked horizontally and peptides shown as markers"""
    # Get proteins with matches
    proteins_with_matches = {pid for pid, matches in mapping.items() if matches}
    
    if not proteins_with_matches:
        print("No peptide matches found in any proteins.")
        return
    
    # Filter proteins and sort them
    filtered_proteins = {pid: protein_dict[pid] for pid in proteins_with_matches}
    filtered_info = {pid: protein_info[pid] for pid in proteins_with_matches}
    
    # Sort proteins by name
    sorted_proteins = sorted(filtered_proteins.keys(), 
                            key=lambda pid: filtered_info[pid]['name'])
    
    # Define protein colors (matching the screenshot)
    protein_colors = {
        'nsp1': '#f2c1d1',   # Light pink
        'nsp4': '#c25b6c',   # Dark pink
        'nsp5': '#b08bc5',   # Purple
        'nsp8': '#f2c1d1',   # Light pink
        'nsp9': '#f2c1d1',   # Light pink
        'E': '#e35946',      # Red
        'N': '#8bc54c',      # Green
        'PB1': '#f2c1d1',    # Light pink
        'PB2': '#c25b6c',    # Dark pink
        'PA': '#b08bc5',     # Purple
        'HA': '#f2c1d1',     # Light pink
        'NA': '#e35946',     # Red
        'M1': '#8bc54c',     # Green
        'NS1': '#5c8bc5',    # Blue
        'NP': '#f2c1d1'      # Light pink
    }
    
    # Setup figure
    fig, ax = plt.subplots(figsize=(16, 8))
    
    # Total width available for all proteins (leaving margins)
    available_width = 0.8
    left_margin = 0.1
    
    # Calculate total length of all proteins for scaling
    total_length = sum(len(filtered_proteins[pid]) for pid in sorted_proteins)
    
    # Calculate positions and widths for each protein
    protein_positions = {}
    x_start = left_margin
    
    for protein_id in sorted_proteins:
        seq_length = len(filtered_proteins[protein_id])
        relative_width = (seq_length / total_length) * available_width
        protein_positions[protein_id] = (x_start, relative_width)
        x_start += relative_width
    
    # Vertical spacing parameters
    peptide_rows = 10  # Number of rows for peptide markers
    row_height = 0.05  # Height of each row as fraction of plot height
    
    # Track peptide positions to handle overlaps
    peptide_y_positions = {}
    
    # Draw peptide matches for each protein
    peptide_colors = ['#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
    
    # First, draw the protein bars at the bottom
    y_protein = 0.20  # Position of protein bars
    protein_height = 0.05
    
    for protein_id in sorted_proteins:
        x_start, width = protein_positions[protein_id]
        gene_name = filtered_info[protein_id]['name']
        
        # Get color for this protein
        protein_color = protein_colors.get(gene_name.lower(), '#cccccc')
        
        # Draw protein rectangle
        protein_rect = patches.Rectangle(
            (x_start, y_protein - protein_height/2),
            width,
            protein_height,
            facecolor=protein_color,
            edgecolor='black',
            alpha=0.8
        )
        ax.add_patch(protein_rect)
        
        # Add protein label
        ax.text(
            x_start + width/2,
            y_protein - 0.05,
            gene_name,
            ha='center',
            va='top',
            fontsize=10,
            fontweight='bold'
        )
    
    # Next, draw the peptide markers above
    for protein_id in sorted_proteins:
        x_start, width = protein_positions[protein_id]
        protein_seq = filtered_proteins[protein_id]
        protein_length = len(protein_seq)
        
        # Create a dictionary to track which rows are occupied
        row_occupancy = [[] for _ in range(peptide_rows)]
        
        for match in sorted(mapping[protein_id], key=lambda m: m['start']):
            # Calculate relative position within the protein space
            rel_start = match['start'] / protein_length
            rel_width = (match['end'] - match['start']) / protein_length
            
            # Calculate absolute position in the plot
            abs_start = x_start + (rel_start * width)
            abs_width = rel_width * width
            
            # Find a row where this peptide fits (no overlap)
            assigned_row = None
            for row_idx in range(peptide_rows):
                overlaps = False
                for occupied_start, occupied_width in row_occupancy[row_idx]:
                    # Check if this peptide overlaps with any existing one in this row
                    if (abs_start < occupied_start + occupied_width and 
                        abs_start + abs_width > occupied_start):
                        overlaps = True
                        break
                
                if not overlaps:
                    assigned_row = row_idx
                    row_occupancy[row_idx].append((abs_start, abs_width))
                    break
            
            # If no row is available, use the last row
            if assigned_row is None:
                assigned_row = peptide_rows - 1
                row_occupancy[assigned_row].append((abs_start, abs_width))
            
            # Calculate y position based on row
            y_pos = 0.3 + (assigned_row * row_height)
            
            # Get a color for this peptide (based on peptide index)
            peptide_idx = hash(match['peptide']) % len(peptide_colors)
            peptide_color = peptide_colors[peptide_idx]
            
            # Draw peptide rectangle
            peptide_rect = patches.Rectangle(
                (abs_start, y_pos - 0.015),
                abs_width,
                0.03,
                facecolor=peptide_color,
                alpha=0.8
            )
            ax.add_patch(peptide_rect)
    
    # Set plot limits and labels
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 0.9)
    
    # Remove axes and ticks
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off')
    
    # Add a title
    plt.suptitle('Peptide Distribution Across Viral Proteins', fontsize=14, y=0.95)
    
    # Add subtle grid lines at protein boundaries
    for x_pos, _ in protein_positions.values():
        if x_pos > left_margin:
            ax.axvline(x=x_pos, color='gray', linestyle='-', alpha=0.3)
    
    # Save figure
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Stacked visualization saved to {output_file}")

def create_detailed_visualization(mapping, protein_dict, protein_info, output_file="peptide_detailed_alignment.png"):
    """Create a detailed visualization similar to the screenshot"""
    # Get proteins with matches
    proteins_with_matches = {pid for pid, matches in mapping.items() if matches}
    
    if not proteins_with_matches:
        print("No peptide matches found in any proteins.")
        return
    
    # Filter proteins and sort them
    filtered_proteins = {pid: protein_dict[pid] for pid in proteins_with_matches}
    filtered_info = {pid: protein_info[pid] for pid in proteins_with_matches}
    
    # Sort proteins by name
    sorted_proteins = sorted(filtered_proteins.keys(), 
                           key=lambda pid: filtered_info[pid]['name'])
    
    # Define protein colors (matching the screenshot)
    protein_colors = {
        'nsp1': '#f2c1d1',   # Light pink
        'nsp4': '#c25b6c',   # Dark pink
        'nsp5': '#b08bc5',   # Purple
        'nsp8': '#f2c1d1',   # Light pink
        'nsp9': '#f2c1d1',   # Light pink
        'E': '#e35946',      # Red
        'N': '#8bc54c',      # Green
        'PB1': '#f2c1d1',    # Light pink
        'PB2': '#c25b6c',    # Dark pink
        'PA': '#b08bc5',     # Purple
        'HA': '#f2c1d1',     # Light pink
        'NA': '#e35946',     # Red
        'M1': '#8bc54c',     # Green
        'NS1': '#5c8bc5',    # Blue
        'NP': '#f2c1d1'      # Light pink
    }
    
    # Setup figure
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8), gridspec_kw={'height_ratios': [3, 1]}, sharex=True)
    
    # Calculate total width for all proteins combined
    total_length = sum(len(filtered_proteins[pid]) for pid in sorted_proteins)
    protein_widths = {pid: len(filtered_proteins[pid])/total_length for pid in sorted_proteins}
    
    # Position mapping for each protein
    protein_positions = {}
    x_start = 0
    
    for protein_id in sorted_proteins:
        width = protein_widths[protein_id]
        protein_positions[protein_id] = (x_start, width)
        x_start += width
    
    # Draw peptides in the upper plot (ax1)
    for i, peptide in enumerate(set(match['peptide'] for pid in sorted_proteins 
                               for match in mapping[pid])):
        # Use a different row for each peptide to avoid overlap
        y_pos = 2 - (i % 4) * 0.5
        
        # Use orange for peptide markers (like in the screenshot)
        peptide_color = '#ff7f0e'  # Orange
        
        # Draw this peptide for each protein where it occurs
        for protein_id in sorted_proteins:
            x_start, width = protein_positions[protein_id]
            protein_length = len(filtered_proteins[protein_id])
            
            # Find matches for this peptide in this protein
            for match in mapping[protein_id]:
                if match['peptide'] == peptide:
                    # Calculate relative position
                    rel_start = match['start'] / protein_length
                    rel_width = (match['end'] - match['start']) / protein_length
                    
                    # Calculate absolute position in plot
                    abs_start = x_start + (rel_start * width)
                    abs_width = rel_width * width
                    
                    # Draw rectangle for this peptide match
                    rect = patches.Rectangle(
                        (abs_start, y_pos - 0.1),
                        abs_width,
                        0.2,
                        facecolor=peptide_color,
                        alpha=0.7
                    )
                    ax1.add_patch(rect)
    
    # Draw protein bars in the lower plot (ax2)
    for protein_id in sorted_proteins:
        x_start, width = protein_positions[protein_id]
        gene_name = filtered_info[protein_id]['name']
        
        # Get color for this protein
        protein_color = protein_colors.get(gene_name.lower(), '#cccccc')
        
        # Draw protein rectangle
        protein_rect = patches.Rectangle(
            (x_start, 0.25),
            width,
            0.5,
            facecolor=protein_color,
            edgecolor='black',
            alpha=0.8
        )
        ax2.add_patch(protein_rect)
        
        # Add protein label
        ax2.text(
            x_start + width/2,
            0.5,
            gene_name,
            ha='center',
            va='center',
            fontsize=10,
            fontweight='bold'
        )
    
    # Set plot limits and labels
    ax1.set_ylim(0, 2.5)
    ax2.set_ylim(0, 1)
    ax1.set_xlim(0, 1)
    
    # Remove ticks and set labels
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax2.set_xticks([])
    ax2.set_yticks([])
    
    # Remove spines
    for spine in ax1.spines.values():
        spine.set_visible(False)
    for spine in ax2.spines.values():
        spine.set_visible(False)
    
    # Add title
    fig.suptitle('Peptide Distribution Across Viral Proteins', fontsize=14, y=0.95)
    
    # Add subtle grid lines at protein boundaries
    for x_pos, _ in protein_positions.values():
        if x_pos > 0:
            ax1.axvline(x=x_pos, color='gray', linestyle='-', alpha=0.3)
            ax2.axvline(x=x_pos, color='gray', linestyle='-', alpha=0.3)
    
    # Save figure
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Detailed visualization saved to {output_file}")
    
def export_csv(mapping, protein_info, csv_file="peptide_positions.csv"):
    """Export peptide mappings to a CSV file"""
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        
        # Write header
        writer.writerow(['peptide', 'protein', 'protein_length', 'start_position', 'end_position'])
        
        # Write data rows
        for protein_id, matches in mapping.items():
            if not matches:
                continue
                
            protein_name = protein_info[protein_id]['name']
            protein_length = protein_info[protein_id]['length']
            
            for match in matches:
                writer.writerow([
                    match['peptide'],
                    protein_name,
                    protein_length,
                    match['start'] + 1,  # Convert to 1-indexed for readability
                    match['end']
                ])
    
    print(f"CSV export saved to {csv_file}")

def main():
    """Main function to run the peptide visualization process"""
    # File paths
    fasta_file = "X31_proteomes.fasta"
    peptides_file = "peptides.txt"
    output_file1 = "peptide_stacked_alignment.png"
    output_file2 = "peptide_detailed_alignment.png"
    csv_file = "peptide_positions.csv"
    
    # Check file existence
    if not os.path.exists(fasta_file):
        print(f"Error: FASTA file '{fasta_file}' not found")
        return
    
    if not os.path.exists(peptides_file):
        print(f"Error: Peptides file '{peptides_file}' not found")
        return
    
    # Parse files
    print(f"Parsing proteins from {fasta_file}")
    protein_dict, protein_info = parse_fasta(fasta_file)
    print(f"Found {len(protein_dict)} proteins")
    
    print(f"Parsing peptides from {peptides_file}")
    peptides = parse_peptides(peptides_file)
    print(f"Found {len(peptides)} peptides")
    
    # Find peptide matches
    print("Finding peptide matches")
    mapping = find_peptide_matches(peptides, protein_dict)
    
    # Count matches per protein
    total_matches = 0
    for protein_id, matches in mapping.items():
        if matches:
            gene_name = protein_info[protein_id]['name']
            match_count = len(matches)
            total_matches += match_count
            print(f"  {gene_name}: {match_count} peptide matches")
    
    print(f"Total: {total_matches} peptide matches")
    
    # Create visualizations
    print("Creating stacked visualization")
    create_stacked_visualization(mapping, protein_dict, protein_info, output_file1)
    
    print("Creating detailed visualization")
    create_detailed_visualization(mapping, protein_dict, protein_info, output_file2)
    
    # Export to CSV
    print("Exporting match positions to CSV")
    export_csv(mapping, protein_info, csv_file)
    
    print("Process completed successfully")

if __name__ == "__main__":
    main()

Parsing proteins from X31_proteomes.fasta
Found 56 proteins
Parsing peptides from peptides.txt
Found 81 peptides
Finding peptide matches
  PB2: 14 peptide matches
  HKx31_HA_translation_frame_+3: 7 peptide matches
  PR8_PA_translation_frame_+1: 4 peptide matches
  NS: 5 peptide matches
  M: 8 peptide matches
  NP: 23 peptide matches
  HA: 7 peptide matches
  PR8_M_translation_frame_+2: 8 peptide matches
  PR8_PB1_translation_frame_+1: 9 peptide matches
  PR8_PB2_translation_frame_+1: 16 peptide matches
  PR8_NS_translation_frame_+1: 6 peptide matches
  PR8_NS_translation_frame_+3: 4 peptide matches
  NA: 1 peptide matches
  PR8_NP_translation_frame_-1: 9 peptide matches
  HKx31_NA_translation_frame_+1: 1 peptide matches
  NS: 4 peptide matches
  HKx31_HA_translation_frame_+1: 2 peptide matches
  PR8_PB2_translation_frame_+3: 1 peptide matches
  PR8_NP_translation_frame_+1: 23 peptide matches
Total: 152 peptide matches
Creating stacked visualization
Stacked visualization saved to peptid

## New 

In [26]:
#!/usr/bin/env python3
"""
Protein Peptide Distribution Visualization

This script visualizes the distribution of peptides across protein positions.
It reads peptide data from a text file and protein sequences from a FASTA file,
then generates publication-quality plots for each protein.

Usage:
    python peptide_visualization.py --peptides peptides.txt --fasta protein_seq.fasta [--mutations N]

Arguments:
    --peptides: Path to peptide list file
    --fasta: Path to protein sequence FASTA file
    --mutations: Number of allowed mutations (default: 0)
"""

import argparse
import csv
import re
import os
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import numpy as np
from Bio import SeqIO

def parse_arguments():
    """Parse command line arguments."""
    parser = argparse.ArgumentParser(description='Visualize peptide distribution across protein positions')
    parser.add_argument('--peptides', required=True, help='Path to peptide list file')
    parser.add_argument('--fasta', required=True, help='Path to protein sequence FASTA file')
    parser.add_argument('--mutations', type=int, default=0, help='Number of allowed mutations (default: 0)')
    parser.add_argument('--output_dir', default='peptide_plots', help='Output directory for plots')
    return parser.parse_args()

def read_peptides(peptide_file):
    """Read peptide data from file."""
    peptides = []
    with open(peptide_file, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)  # Skip header
        for row in reader:
            if not row:
                continue
            peptide_data = {
                'peptide': row[0],
                'source_protein': row[1].strip(),
                'start': int(row[2]) if len(row) > 2 and row[2].isdigit() else None,
                'end': int(row[3]) if len(row) > 3 and row[3].isdigit() else None,
                'matched_sequence': row[4] if len(row) > 4 else '',
                'mutations': row[5] if len(row) > 5 else ''
            }
            peptides.append(peptide_data)
    return peptides

def read_protein_sequences(fasta_file):
    """Read protein sequences from FASTA file."""
    proteins = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        protein_id = record.id
        # Handle cases where description contains more information
        description = record.description
        if '(' in description and ')' in description:
            match = re.search(r'\((.*?)\)', description)
            if match:
                protein_id = match.group(1)
        proteins[protein_id] = str(record.seq)
    return proteins

def hamming_distance(seq1, seq2):
    """Calculate Hamming distance between two sequences."""
    return sum(c1 != c2 for c1, c2 in zip(seq1, seq2))

def find_peptide_positions(protein_seq, peptide_seq, max_mutations=0):
    """Find positions of peptide in protein with allowed mutations."""
    positions = []
    peptide_len = len(peptide_seq)
    
    for i in range(len(protein_seq) - peptide_len + 1):
        protein_subseq = protein_seq[i:i+peptide_len]
        distance = hamming_distance(protein_subseq, peptide_seq)
        if distance <= max_mutations:
            positions.append({
                'start': i + 1,  # 1-based indexing
                'end': i + peptide_len,
                'matched_sequence': protein_subseq,
                'mutations': distance
            })
    
    return positions

def group_peptides_by_protein(peptides):
    """Group peptides by source protein."""
    protein_peptides = defaultdict(list)
    for peptide in peptides:
        protein_name = peptide['source_protein']
        protein_peptides[protein_name].append(peptide)
    return protein_peptides

def extract_protein_id(protein_name):
    """Extract protein ID from protein name."""
    match = re.search(r'\((.*?)\)', protein_name)
    if match:
        return match.group(1)
    return protein_name

def map_unmapped_peptides(peptides, protein_sequences, max_mutations):
    """Map peptides that don't have start/end positions using sequence matching."""
    mapped_peptides = []
    
    for peptide in peptides:
        if peptide['start'] is not None and peptide['end'] is not None:
            mapped_peptides.append(peptide)
            continue
        
        protein_id = extract_protein_id(peptide['source_protein'])
        
        if protein_id in protein_sequences:
            protein_seq = protein_sequences[protein_id]
            positions = find_peptide_positions(protein_seq, peptide['peptide'], max_mutations)
            
            if positions:
                for pos in positions:
                    new_peptide = peptide.copy()
                    new_peptide['start'] = pos['start']
                    new_peptide['end'] = pos['end']
                    new_peptide['matched_sequence'] = pos['matched_sequence']
                    new_peptide['mutations'] = pos['mutations']
                    mapped_peptides.append(new_peptide)
            else:
                # Keep the peptide even if we couldn't map it
                mapped_peptides.append(peptide)
        else:
            # Keep the peptide even if we couldn't find the protein
            mapped_peptides.append(peptide)
    
    return mapped_peptides

def plot_peptide_distribution(protein_name, peptides, protein_length, output_dir):
    """Create a publication-quality plot for peptide distribution of a protein."""
    plt.figure(figsize=(12, 6))
    
    # Create a colormap for different peptides
    num_peptides = len(peptides)
    colors = sns.color_palette("husl", num_peptides)
    
    # Set up the plot
    ax = plt.gca()
    ax.set_xlim(0, protein_length + 50)  # Add some padding
    ax.set_ylim(-1, num_peptides + 1)  # Add some padding
    
    # Draw the protein backbone
    plt.plot([0, protein_length], [0, 0], 'k-', linewidth=2)
    
    # Add position markers
    for i in range(0, protein_length + 1, 100):
        plt.plot([i, i], [-0.2, 0.2], 'k-')
        plt.text(i, -0.5, str(i), ha='center')
    
    # Sort peptides by start position
    peptides_sorted = sorted(peptides, key=lambda x: x.get('start', 0) if x.get('start') is not None else 0)
    
    # Draw peptides
    y_positions = {}
    
    for i, peptide in enumerate(peptides_sorted):
        if peptide['start'] is None or peptide['end'] is None:
            continue
        
        start = peptide['start']
        end = peptide['end']
        mutations = peptide['mutations'] if peptide['mutations'] else 0
        
        # Find a suitable y-position (avoid overlaps)
        y = 1
        while any(start <= y_positions.get((other_start, other_end), (0, 0))[1] <= end
                and y == y_positions.get((other_start, other_end), (0, 0))[0]
                for other_start, other_end in y_positions.keys()):
            y += 1
        
        y_positions[(start, end)] = (y, end)
        
        # Draw peptide
        rect = patches.Rectangle((start, y - 0.4), end - start, 0.8, 
                                 linewidth=1, edgecolor='black', 
                                 facecolor=colors[i % len(colors)], 
                                 alpha=0.7)
        ax.add_patch(rect)
        
        # Add peptide label
        plt.text(start + (end - start) / 2, y, 
                 peptide['peptide'], 
                 ha='center', va='center', 
                 fontsize=8, fontweight='bold')
        
        # Add mutations information if any
        if mutations:
            plt.text(end + 5, y, f"{mutations} mut", 
                     ha='left', va='center', 
                     fontsize=7, style='italic', color='red')
    
    # Add labels and title
    plt.title(f"Peptide Distribution for {protein_name}", fontsize=14, fontweight='bold')
    plt.xlabel("Amino Acid Position", fontsize=12)
    plt.ylabel("Peptides", fontsize=12)
    plt.tick_params(axis='y', which='both', left=False, labelleft=False)
    
    # Add a legend
    legend_elements = [patches.Patch(facecolor=colors[i % len(colors)], 
                                     edgecolor='black',
                                     label=peptides_sorted[i]['peptide'])
                       for i in range(len(peptides_sorted))
                       if peptides_sorted[i]['start'] is not None]
    
    plt.legend(handles=legend_elements, 
               loc='upper center', 
               bbox_to_anchor=(0.5, -0.15),
               fancybox=True, shadow=True, 
               ncol=min(5, len(legend_elements)))
    
    # Make it look nice
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    sns.despine(left=True)
    
    # Save the plot
    os.makedirs(output_dir, exist_ok=True)
    safe_protein_name = re.sub(r'[^\w\-_\.]', '_', protein_name)
    output_file = os.path.join(output_dir, f"{safe_protein_name}_peptide_distribution.png")
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.savefig(output_file.replace('.png', '.pdf'), format='pdf', bbox_inches='tight')
    
    plt.close()
    return output_file

def create_protein_coverage_plot(protein_name, peptides, protein_length, output_dir):
    """Create a coverage plot showing which regions of the protein are covered by peptides."""
    coverage = np.zeros(protein_length)
    
    for peptide in peptides:
        if peptide['start'] is None or peptide['end'] is None:
            continue
        
        start = peptide['start'] - 1  # Convert to 0-based for array indexing
        end = peptide['end'] - 1      # Convert to 0-based for array indexing
        
        coverage[start:end+1] += 1
    
    plt.figure(figsize=(12, 4))
    
    # Set up the plot
    ax = plt.gca()
    ax.set_xlim(0, protein_length)
    
    # Create a heat map of coverage
    plt.imshow([coverage], aspect='auto', cmap='viridis', 
               extent=[0, protein_length, 0, 1], 
               interpolation='nearest')
    
    # Add a color bar
    cbar = plt.colorbar(orientation='vertical', pad=0.01)
    cbar.set_label('Number of Peptides Covering Position')
    
    # Add position markers
    for i in range(0, protein_length + 1, 100):
        plt.axvline(x=i, color='gray', linestyle='--', alpha=0.5)
        plt.text(i, 1.1, str(i), ha='center', transform=ax.get_xaxis_transform())
    
    # Add labels and title
    plt.title(f"Peptide Coverage for {protein_name}", fontsize=14, fontweight='bold')
    plt.xlabel("Amino Acid Position", fontsize=12)
    plt.tick_params(axis='y', which='both', left=False, labelleft=False)
    
    # Save the plot
    os.makedirs(output_dir, exist_ok=True)
    safe_protein_name = re.sub(r'[^\w\-_\.]', '_', protein_name)
    output_file = os.path.join(output_dir, f"{safe_protein_name}_coverage.png")
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.savefig(output_file.replace('.png', '.pdf'), format='pdf', bbox_inches='tight')
    
    plt.close()
    return output_file

def main():
    # Parse arguments
    args = parse_arguments()
    
    # Read peptide data
    peptides = read_peptides(args.peptides)
    
    # Read protein sequences
    protein_sequences = read_protein_sequences(args.fasta)
    
    # Map peptides that don't have positions
    peptides = map_unmapped_peptides(peptides, protein_sequences, args.mutations)
    
    # Group peptides by protein
    protein_peptides = group_peptides_by_protein(peptides)
    
    # Create output directory
    os.makedirs(args.output_dir, exist_ok=True)
    
    # Generate plots for each protein
    for protein_name, peptides in protein_peptides.items():
        print(f"Processing {protein_name}...")
        
        # Extract protein ID and get sequence length
        protein_id = extract_protein_id(protein_name)
        
        # If protein sequence is available, use its length
        if protein_id in protein_sequences:
            protein_length = len(protein_sequences[protein_id])
        else:
            # Otherwise, estimate from the maximum end position of peptides
            max_end = max([p['end'] for p in peptides if p['end'] is not None], default=0)
            protein_length = max(max_end, 700)  # Use a default minimum length
        
        # Create the distribution plot
        dist_plot = plot_peptide_distribution(protein_name, peptides, protein_length, args.output_dir)
        print(f"  Created distribution plot: {dist_plot}")
        
        # Create the coverage plot
        cov_plot = create_protein_coverage_plot(protein_name, peptides, protein_length, args.output_dir)
        print(f"  Created coverage plot: {cov_plot}")
    
    print(f"\nAll plots saved to {args.output_dir}")

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] --peptides PEPTIDES --fasta FASTA
                             [--mutations MUTATIONS] [--output_dir OUTPUT_DIR]
ipykernel_launcher.py: error: the following arguments are required: --peptides


SystemExit: 2

/Users/sanjay/Monash/Master_thesis/lab_work/Li_Lab/ProtPeptigram/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
